<a href="https://colab.research.google.com/github/cpappas18/Articulaite/blob/main/Random_Forest_Classifier_on_one_hot_encoded_datasets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.ensemble import RandomForestClassifier
import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score, KFold
import pandas as pd
from sklearn.model_selection import train_test_split


In [ ]:
df = pd.read_csv('all_disorders_remove_std_onehot (1).csv')

In [ ]:
df

,Unnamed: 0,Control,Parkinsons,ALS,Cerebral_palsy,Gender,MDVP:Jitter(%),MDVP:Jitter(Abs),Jitter:PPQ5,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,DFA
0,0,0,1,0,0,0,0.014546,-0.000127,0.001828,0.063985,-0.181563,-0.409024,-0.076295,-0.151238,-0.220074,-1.024965
1,1,0,1,0,0,0,-0.046874,-0.000417,-0.026584,-0.072149,-0.263543,-0.594171,-0.149456,-0.189609,-0.215807,-0.867248
2,2,0,1,0,0,0,-0.016164,-0.000284,-0.016717,-0.034960,-0.316428,-0.586910,-0.192299,-0.226523,-0.252313,-1.067120
3,3,0,1,0,0,0,-0.008189,-0.000250,-0.007189,-0.042196,-0.220356,-0.056884,-0.131001,-0.158362,-0.172506,-0.740206
4,4,0,1,0,0,0,-0.040936,-0.000360,-0.029987,-0.091837,-0.312185,-0.605061,-0.201361,-0.212762,-0.195263,-0.901492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6679,6679,1,0,0,0,1,-32.217900,-20.908653,-23.684420,-31.694701,-22.688858,-7.370397,-13.781598,-10.612545,-23.216192,-23.194185
6680,6680,1,0,0,0,1,-1.059601,18.348724,-2.716940,-3.364087,-1.038437,-2.693716,0.433780,4.189205,-4.254203,-1.252287
6681,6681,1,0,0,0,1,35.218278,-0.558297,42.491249,30.454924,-16.763825,1.337682,-16.327157,-16.064503,-18.499916,-14.613771
6682,6682,1,0,0,0,1,2.792539,-3.296699,1.430399,7.347828,-18.280037,-2.884069,-19.637068,-25.929862,-28.065778,-19.527662


In [ ]:
#get rid of unnamed column 
if df.columns[[0]] == 'Unnamed: 0':
    i = df.drop(df.columns[[0]], axis=1, inplace=True) 

In [ ]:
#getting features and labels from original dataset
features = df.drop(['Control', 'Parkinsons', 'ALS', 'Cerebral_palsy'], axis = 1)
labels = df.drop(['Gender', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'Jitter:PPQ5', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 'Shimmer:APQ11', 'DFA'],axis = 1)

In [ ]:
x = features 
y = labels 

In [ ]:
#splitting the train test split 
x_train,x_test,y_train,y_test = train_test_split(x, y, test_size=0.2, random_state=0, stratify=y)

Without SMOTE

In [ ]:
model_ns = RandomForestClassifier(n_estimators=1000, random_state = 1)
model_ns.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [ ]:
y_test.value_counts() #PD is 96% our data

Control  Parkinsons  ALS  Cerebral_palsy
0        1           0    0                 1288
1        0           0    0                   43
0        0           1    0                    3
                     0    1                    3
dtype: int64

Getting metrics 

In [ ]:
y_pred_ns = model_ns.predict(x_test)
from sklearn import metrics
print(metrics.classification_report(y_pred_ns, y_test))

In [ ]:
print(accuracy_score(y_test, y_pred_ns)*100)

97.08302169035153


In [ ]:
y_test_list = y_test.values.tolist()
y_pred_ns_list = y_pred_ns.tolist()

In [ ]:
print(y_test_list[0] == y_pred_ns_list[0])

True


In [ ]:
#function to get TP, TN, FP, FN
def perf_measure(y_actual, y_hat):
    
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        #actually positive and predicts positive 
        if y_actual[i]==y_hat[i] and y_actual[i] != [1,0,0,0]:
           TP += 1
        #actually negative but says positive 
        if y_actual[i]==[1,0,0,0] and y_hat[i] != [1,0,0,0]:
           FP += 1
        #actually negative, prdicts negative 
        if y_actual[i]==y_hat[i]==[1,0,0,0]:
           TN += 1
        #actually postive, predicts negative 
        if y_actual[i] != [1,0,0,0] and y_hat[i] == [1,0,0,0]:
           FN += 1
    return TP, FP, FN, TN

In [ ]:
TP,FP,FN,TN = perf_measure(y_test_list, y_pred_ns_list)
print(TP, TN, FP, FN)

1285 30 7 13


In [ ]:
kfold = KFold(n_splits=10, shuffle=True)
kf_cv_scores = cross_val_score(model_ns, x_train, y_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

K-fold CV average score: 0.97


With SMOTE

In [ ]:
#concatenate the training dataset together to perform SMOTE on it 
pre_smote_training_dataset = pd.concat([y_train, x_train], axis=1, join='inner')

In [ ]:
pre_smote_training_dataset.reset_index(inplace = True)

In [ ]:
pre_smote_training_dataset.drop(['index'],axis = 1, inplace = True)

In [ ]:
pre_smote_training_dataset

,Control,Parkinsons,ALS,Cerebral_palsy,Gender,MDVP:Jitter(%),MDVP:Jitter(Abs),Jitter:PPQ5,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,DFA
0,0,1,0,0,1,0.007929,0.000475,0.009994,0.025618,1.942476,3.816662,0.727164,1.456949,2.300407,-0.283066
1,0,1,0,0,0,-0.031434,-0.000120,-0.017908,-0.059528,-0.060640,-0.140381,-0.055862,-0.057334,-0.057615,-0.733357
2,0,1,0,0,1,-0.023969,-0.000284,-0.013484,-0.026715,-0.283545,-0.655886,-0.146490,-0.186533,-0.267642,-0.076924
3,0,1,0,0,0,-0.043141,-0.000313,-0.025223,-0.063230,-0.093068,-0.169424,-0.021094,-0.093439,-0.134578,-0.501555
4,0,1,0,0,1,0.048820,0.000085,0.024285,0.080980,-0.189443,-0.263812,-0.119302,-0.158038,-0.229240,0.243528
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5342,0,1,0,0,1,-0.018370,-0.000023,-0.010592,-0.049768,0.017703,0.084698,0.015652,0.002732,0.050796,-0.228564
5343,0,1,0,0,1,-0.026683,-0.000292,-0.012293,-0.030753,0.097409,0.208129,0.088648,0.061503,0.022034,0.841697
5344,0,1,0,0,1,0.010305,-0.000140,-0.003276,-0.017123,-0.024121,-0.082296,-0.037901,-0.012163,0.023614,-0.547954
5345,0,1,0,0,0,0.053740,0.000449,0.033132,0.099154,0.330012,0.734525,0.240244,0.230206,0.152412,1.376393


SMOTE FUNCTION

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter
from imblearn.over_sampling import SMOTE
from numpy import where, random


def smote_multiclass(df, num_samples_dict, one_hot_encoded):
  """
  Performs data augmentation and/or downsampling on a dataframe with multiple classes.
  Supports both numerical and one-hot encoded class values.
  The SMOTE algorithm is used for augmentation and random sampling is used for downsampling of the specified classes.
  Keyword arguments:
  df -- the dataframe to manipulate
  num_samples_dict -- a dictionary of the form {0: num_desired_samples, 1: num_desired_samples, ...} that indicates the
                      number of samples that the returned dataframe should have in each class
  one_hot_encoded -- Boolean value indicating if the dataframe is one-hot encoded
  """
  # change one hot encoded dataset into numerical labels
  if one_hot_encoded:
    df.insert(loc=0, column='Status', value=np.nan)
    for i, row in df.iterrows():
      if df.iloc[i]['Control'] == 1:
          status = 0
      elif df.iloc[i]['Parkinsons'] == 1:
          status = 1
      elif df.iloc[i]['ALS'] == 1:
          status = 2
      elif df.iloc[i]['Cerebral_palsy'] == 1:
          status = 3
      df.at[i,'Status'] = status
    df = df.drop(columns=['Control', 'Parkinsons', 'ALS', 'Cerebral_palsy'])


  if num_samples_dict is not None:
    strategy = num_samples_dict

    # downsampling classes if needed
    counter = Counter(df.Status)
    class_samples = []
    for i in range(4):
      if counter[i] > num_samples_dict[i]: # class i needs to be downsampled
        class_i = df.loc[df['Status'] == i]
        class_i_sample = class_i.sample(n=num_samples_dict[i])
        class_samples.append(class_i_sample)
      else: # class i does not need downsampling
        class_i = df.loc[df['Status'] == i]
        class_samples.append(class_i)

    df = pd.concat(class_samples).reset_index(drop=True)

  else:
    strategy = 'auto'

  y = df.Status # labels
  x = df.drop(['Status', 'Gender'], axis=1) # features only, Gender also removed because it must be binary
  gender = df.Gender

  oversample = SMOTE(sampling_strategy=strategy)
  x_oversampled, y_oversampled = oversample.fit_resample(x, y)

  # get sample counts for augmented dataset
  counter = Counter(y_oversampled)

  if num_samples_dict is not None and (counter[0] != num_samples_dict[0] or counter[1] != num_samples_dict[1]
                                       or counter[2] != num_samples_dict[2] or counter[3] != num_samples_dict[3]):
    print("Sampling failed")
  elif num_samples_dict is None and (counter[0] != counter[1] or counter[1] != counter[2] or counter[2] != counter[3]):
    print("Sampling failed")

  # regenerate dataframe with new samples
  df_concat = pd.concat([pd.DataFrame(y_oversampled), pd.DataFrame(gender), pd.DataFrame(x_oversampled)], axis=1)
  df_concat.columns = df.columns

  # fill missing gender values with a random choice of 0 or 1
  for row in df_concat.loc[df_concat.Gender.isnull(), 'Gender'].index:
    df_concat.at[row, 'Gender'] = random.randint(0, 2)

  # change numerical class values back into one-hot encoded values
  if one_hot_encoded:
    one_hot = pd.get_dummies(df_concat['Status'])
    one_hot.rename(columns={0:'Control', 1:'Parkinsons', 2:'ALS', 3:'Cerebral_palsy'}, inplace=True)
    df_concat = df_concat.drop(columns=['Status'], axis=1)
    df_concat = pd.concat([one_hot, df_concat], axis=1)

  return df_concat


In [ ]:
#doing smote on the training data
smoted_training_data = smote_multiclass(pre_smote_training_dataset, num_samples_dict= {0: 6000, 1: 6000, 2: 6000, 3:6000}, one_hot_encoded=True)

In [ ]:
smoted_training_data

,Control,Parkinsons,ALS,Cerebral_palsy,Gender,MDVP:Jitter(%),MDVP:Jitter(Abs),Jitter:PPQ5,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,DFA
0,1,0,0,0,0.0,-0.078772,-0.000606,-0.043938,-0.128857,1.547883,2.981690,0.855032,1.173132,1.174732,0.063334
1,1,0,0,0,1.0,-0.067404,-0.000486,-0.031348,-0.108327,1.417565,3.007102,1.126916,0.497833,0.670130,-0.103644
2,1,0,0,0,0.0,-0.084880,-0.000636,-0.047000,-0.132895,-0.118980,-0.289224,-0.050095,-0.052963,-0.123832,0.662468
3,1,0,0,0,1.0,-0.073852,-0.000546,-0.041046,-0.124818,0.107259,0.164565,0.116825,0.091779,-0.036439,0.480249
4,1,0,0,0,1.0,-0.075379,-0.000550,-0.038153,-0.120611,1.354224,2.843738,0.777092,0.929468,1.037717,0.580667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0,0,0,1,0.0,0.667727,-5.661660,0.779186,2.734428,0.790042,-2.487118,-6.295278,-1.924544,2.308438,-6.198002
23996,0,0,0,1,1.0,-28.298963,-17.264532,-22.241628,-26.527949,-0.345857,-2.389292,-3.562198,8.197130,15.293083,-18.696573
23997,0,0,0,1,1.0,1.209999,-5.326130,1.131166,3.320462,1.236034,-2.476562,-6.239357,-1.717869,3.117656,-6.376109
23998,0,0,0,1,0.0,4.525846,7.110670,1.324369,7.941190,13.097747,2.615880,17.406345,18.113278,19.577693,0.180993


In [ ]:
#getting features and labels from the smoted training set 
f = smoted_training_data.drop(['Control', 'Parkinsons', 'ALS', 'Cerebral_palsy'], axis = 1)
l = smoted_training_data.drop(['Gender', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'Jitter:PPQ5', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 'Shimmer:APQ11', 'DFA'],axis = 1)

In [ ]:
x_trainer = f 
y_trainer = l

In [ ]:
model = RandomForestClassifier()

Gridsearch to get the best hyperparameters 

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid =  {'bootstrap': [True], 'max_depth': [10, 20,50],'max_features': ['auto'],'min_samples_leaf': [1, 2, 4],'min_samples_split': [2, 5, 10],'n_estimators': [200, 400, 600, 800, 1000]
}
grid_search = GridSearchCV(estimator = model, param_grid = param_grid, 
                          cv = 3, n_jobs = -1, verbose = 2)
grid_search.fit(x_trainer, y_trainer)
grid_search.best_params_

Fitting 3 folds for each of 135 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   32.3s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 405 out of 405 | elapsed:  6.1min finished


{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1000}

Final Optimal Parameters:
{'bootstrap': True,
 'max_depth': 10,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [ ]:
#training the model with smoted training data
model = RandomForestClassifier(n_estimators=1000, max_depth=10, bootstrap=True, max_features='auto', min_samples_leaf=1, min_samples_split=2)
model.fit(x_trainer, y_trainer)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Getting metrics 

In [ ]:
ypred = model.predict(x_test)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(ypred, y_test))

              precision    recall  f1-score   support

           0       0.91      0.31      0.47       124
           1       0.93      1.00      0.97      1207
           2       1.00      0.75      0.86         4
           3       0.67      1.00      0.80         2

   micro avg       0.93      0.93      0.93      1337
   macro avg       0.88      0.77      0.77      1337
weighted avg       0.93      0.93      0.92      1337
 samples avg       0.93      0.93      0.93      1337



In [ ]:
print(accuracy_score(y_test, ypred)*100)

93.34330590875093


In [ ]:
y_test_list_smote = y_test.values.tolist()
ypred_list_smote = ypred.tolist()

In [ ]:
TP, FP, FN, TN = perf_measure(y_test_list_smote, ypred_list_smote)

In [ ]:
print(TP, FP, FN, TN)

1209 4 85 39


In [ ]:
kfold = KFold(n_splits=5, shuffle=True)
kf_cv_scores = cross_val_score(model, x_trainer, y_trainer, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

K-fold CV average score: 0.98


***Median Dataframe***

In [ ]:
data = pd.read_csv('all_disorders_median_std_onehot.csv')

In [ ]:
#get rid of unnamed column 
if data.columns[[0]] == 'Unnamed: 0':
    data.drop(data.columns[[0]], axis=1, inplace=True)

In [ ]:
data

,Control,Parkinsons,ALS,Cerebral_palsy,Gender,MDVP:Jitter(%),MDVP:Jitter(Abs),Jitter:PPQ5,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,DFA
0,0,1,0,0,0,0.037775,0.000503,-0.011752,0.051462,0.098145,0.313568,0.047848,0.149447,-0.102311,1.560809
1,0,1,0,0,0,0.069989,0.000679,-0.011752,0.100932,0.372398,1.049795,0.209514,0.380995,-0.102311,1.602059
2,0,1,0,0,0,0.084345,0.000855,-0.011752,0.142417,0.231999,0.519712,0.145493,0.270893,-0.102311,1.658218
3,0,1,0,0,0,0.075066,0.000855,-0.011752,0.120199,0.272358,0.648551,0.173852,0.295415,-0.102311,1.599274
4,0,1,0,0,0,0.125313,0.001207,-0.011752,0.200219,0.417743,0.895187,0.269969,0.432209,-0.102311,1.640651
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7114,1,0,0,0,1,-33.243298,-21.578308,-24.440050,-32.691854,-23.328592,-7.466749,-14.202938,-10.933402,-23.923671,-23.417656
7115,1,0,0,0,1,-1.091966,18.936385,-2.803416,-3.468182,-1.064890,-2.724605,0.447177,4.317949,-4.380995,-1.267387
7116,1,0,0,0,1,36.342110,-0.576181,43.847541,31.416897,-17.235723,1.363223,-16.826345,-16.550963,-19.062965,-14.755758
7117,1,0,0,0,1,2.882944,-3.402288,1.476280,7.581402,-18.794885,-2.917623,-20.237481,-26.715980,-28.921770,-19.716314


In [ ]:
features2 = data.drop(['Control', 'Parkinsons', 'ALS', 'Cerebral_palsy'], axis = 1)
labels2 = data.drop(['Gender', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'Jitter:PPQ5', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 'Shimmer:APQ11', 'DFA'],axis = 1)

In [ ]:
X = features2
Y = labels2

In [ ]:
x2_train,x2_test,y2_train,y2_test = train_test_split(X, Y, test_size=0.2, random_state=1, stratify=Y)

Without SMOTE 

In [ ]:
model2_ns = RandomForestClassifier(n_estimators=1000, random_state = 1)
model2_ns.fit(x2_train, y2_train)

Getting metrics 

In [ ]:
from sklearn import metrics
y2_pred_ns = model2_ns.predict(x2_test)
print(metrics.classification_report(y2_pred_ns, y2_test))

              precision    recall  f1-score   support

           0       0.65      0.85      0.74        59
           1       0.99      0.98      0.99      1358
           2       1.00      0.75      0.86         4
           3       0.33      1.00      0.50         1

   micro avg       0.97      0.97      0.97      1422
   macro avg       0.74      0.89      0.77      1422
weighted avg       0.98      0.97      0.98      1422
 samples avg       0.97      0.97      0.97      1422



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y2_pred_ns = model2_ns.predict(x2_test)
from sklearn.metrics import accuracy_score
accuracy_score(y2_test, y2_pred_ns)*100

In [ ]:
y2_test_list = y2_test.values.tolist()
y2_pred_ns_list = y2_pred_ns.tolist()
TP,TN,FP,FN = perf_measure(y2_test_list,y2_pred_ns_list )

In [ ]:
print(TP, TN, FP, FN)

1336 9 27 50


In [ ]:
kfold = KFold(n_splits=5, shuffle=True)
kf_cv_scores = cross_val_score(model2_ns, x2_train, y2_train, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

K-fold CV average score: 0.98


With SMOTE

In [ ]:
pre_smote_training_dataset_median = pd.concat([y2_train, x2_train], axis=1, join='inner')

In [ ]:
pre_smote_training_dataset_median

,Control,Parkinsons,ALS,Cerebral_palsy,Gender,MDVP:Jitter(%),MDVP:Jitter(Abs),Jitter:PPQ5,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,DFA
4412,0,1,0,0,0,0.037424,0.000513,0.015636,0.019177,0.075550,0.221540,-0.018381,0.040012,0.249007,0.822333
3043,0,1,0,0,0,0.047929,0.000278,0.027399,0.100411,0.250231,1.035070,0.131398,0.163793,0.190535,-0.006756
2361,0,1,0,0,0,-0.014223,0.000107,-0.000867,-0.009290,-0.146657,-0.374803,-0.069836,-0.075762,-0.140261,0.507213
619,0,1,0,0,0,-0.052740,-0.000414,-0.027377,-0.077680,-0.504900,-1.088943,-0.277523,-0.321657,-0.428384,-0.524424
4082,0,1,0,0,0,0.005035,-0.000118,-0.000691,0.007026,-0.061109,0.048527,-0.056251,-0.048236,0.037434,-0.228878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1906,0,1,0,0,0,0.003459,-0.000101,-0.001394,-0.011199,-0.185769,-0.363760,-0.122310,-0.109460,-0.068922,-0.808772
2755,0,1,0,0,0,0.499979,0.003654,0.206296,0.916922,0.979650,2.338191,0.575131,0.425870,0.634202,1.009983
4543,0,1,0,0,1,-0.041535,-0.000417,-0.022461,-0.069174,-0.262747,-0.477875,-0.133857,-0.165845,-0.238636,-0.489660
5508,0,1,0,0,0,0.059659,0.000346,0.023185,0.095030,-0.060018,-0.139211,-0.089705,-0.164678,-0.178373,-0.260916


In [ ]:
pre_smote_training_dataset_median.reset_index(inplace = True)

In [ ]:
pre_smote_training_dataset_median.drop(['index'],axis = 1, inplace = True)

In [ ]:
pre_smote_training_dataset_median

,Control,Parkinsons,ALS,Cerebral_palsy,Gender,MDVP:Jitter(%),MDVP:Jitter(Abs),Jitter:PPQ5,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,DFA
0,0,1,0,0,0,0.037424,0.000513,0.015636,0.019177,0.075550,0.221540,-0.018381,0.040012,0.249007,0.822333
1,0,1,0,0,0,0.047929,0.000278,0.027399,0.100411,0.250231,1.035070,0.131398,0.163793,0.190535,-0.006756
2,0,1,0,0,0,-0.014223,0.000107,-0.000867,-0.009290,-0.146657,-0.374803,-0.069836,-0.075762,-0.140261,0.507213
3,0,1,0,0,0,-0.052740,-0.000414,-0.027377,-0.077680,-0.504900,-1.088943,-0.277523,-0.321657,-0.428384,-0.524424
4,0,1,0,0,0,0.005035,-0.000118,-0.000691,0.007026,-0.061109,0.048527,-0.056251,-0.048236,0.037434,-0.228878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5690,0,1,0,0,0,0.003459,-0.000101,-0.001394,-0.011199,-0.185769,-0.363760,-0.122310,-0.109460,-0.068922,-0.808772
5691,0,1,0,0,0,0.499979,0.003654,0.206296,0.916922,0.979650,2.338191,0.575131,0.425870,0.634202,1.009983
5692,0,1,0,0,1,-0.041535,-0.000417,-0.022461,-0.069174,-0.262747,-0.477875,-0.133857,-0.165845,-0.238636,-0.489660
5693,0,1,0,0,0,0.059659,0.000346,0.023185,0.095030,-0.060018,-0.139211,-0.089705,-0.164678,-0.178373,-0.260916


In [ ]:
smoted_training_data_median = smote_multiclass(pre_smote_training_dataset_median, num_samples_dict= {0: 6000, 1: 6000, 2: 6000, 3:6000}, one_hot_encoded=True)

In [ ]:
smoted_training_data_median

,Control,Parkinsons,ALS,Cerebral_palsy,Gender,MDVP:Jitter(%),MDVP:Jitter(Abs),Jitter:PPQ5,Jitter:DDP,MDVP:Shimmer,MDVP:Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,DFA
0,1,0,0,0,0.0,-0.074975,-0.000609,-0.043704,-0.129753,0.038776,0.089019,0.025941,0.047519,0.075872,1.077954
1,1,0,0,0,0.0,-0.082679,-0.000648,-0.044231,-0.129232,0.269241,0.611740,0.219024,0.167964,0.111379,1.406609
2,1,0,0,0,0.0,-0.076026,-0.000623,-0.047566,-0.134787,-0.174238,-0.393209,-0.112290,-0.099951,-0.093842,0.359002
3,1,0,0,0,0.0,-0.089682,-0.000685,-0.048444,-0.134787,-0.223946,-0.518367,-0.119083,-0.090442,-0.213228,-0.504170
4,1,0,0,0,0.0,-0.069023,-0.000567,-0.042826,-0.117950,-0.076691,-0.194428,-0.009041,-0.064919,-0.123811,1.440497
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,0,0,0,1,1.0,-14.212506,-6.855520,-10.274190,-11.652282,-8.263243,-1.409916,-12.531532,-10.365914,-7.445041,-9.100426
23996,0,0,0,1,0.0,-6.482710,-12.425568,-4.376676,-5.210752,-4.349429,1.949585,-10.604802,-7.634012,-4.543912,-0.331547
23997,0,0,0,1,1.0,-14.115405,1.385679,-10.635964,-13.350614,-10.303957,-1.222654,-13.882467,-11.156465,-3.777064,-11.772905
23998,0,0,0,1,1.0,-8.300057,-11.237979,-10.139267,-5.228933,-9.055826,-4.753947,-6.203100,-8.865617,-4.395439,-12.180905


In [ ]:
f2 = smoted_training_data_median.drop(['Control', 'Parkinsons', 'ALS', 'Cerebral_palsy'], axis = 1)
l2 = smoted_training_data_median.drop(['Gender', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'Jitter:PPQ5', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 'Shimmer:APQ11', 'DFA'],axis = 1)

In [ ]:
x_trainer_median = f2 
y_trainer_median = l2

In [ ]:
model2 = RandomForestClassifier(n_estimators=1000, random_state = 0)
model2.fit(x_trainer_median, y_trainer_median)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)

In [ ]:
y_pred = model2.predict(x2_test)

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_pred, y2_test))

In [ ]:
print(accuracy_score(y2_test, y_pred)*100)

In [ ]:
y_pred_list = y_pred.tolist()
y2_test_list = y2_test.values.tolist()
TP, FP, FN, TN = perf_measure(y2_test_list, y_pred_list)

In [ ]:
print(TP, FP, TN, FN)

1295 13 64 51


In [ ]:
kfold = KFold(n_splits=5, shuffle=True)
kf_cv_scores = cross_val_score(model2, x_trainer_median, y_trainer_median, cv=kfold )
print("K-fold CV average score: %.2f" % kf_cv_scores.mean())

K-fold CV average score: 0.99
